In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.pipeline import FeatureUnion
import string


In [2]:
sklearn_random = 13

In [3]:
len_dataset = 2000

In [4]:
path_training_small_truth = "../data/pan20-authorship-verification-training-small/pan20-authorship-verification-training-small-truth.jsonl"
path_training_small = "../data/pan20-authorship-verification-training-small/pan20-authorship-verification-training-small.jsonl"

In [5]:
df_texts = pd.read_json(path_training_small, lines=True)

In [6]:
df_truth = pd.read_json(path_training_small_truth, lines=True)

In [7]:
df_join_training_data = pd.concat([df_truth, df_texts], axis=1).reindex(df_truth.index)

In [8]:
df_join_training_data.head()

,id,same,authors,id,fandoms,pair
0,6cced668-6e51-5212-873c-717f2bc91ce6,True,"[1446633, 1446633]",6cced668-6e51-5212-873c-717f2bc91ce6,"[Guardians of Ga'Hoole, Hetalia - Axis Powers]","[I shift a bit, warily letting my eyes dart fr..."
1,3c6c188a-db28-59aa-8c09-3d0f799ff579,True,"[1446633, 1446633]",3c6c188a-db28-59aa-8c09-3d0f799ff579,"[Guardians of Ga'Hoole, Warriors]","[I shift a bit, warily letting my eyes dart fr..."
2,b0cfa94f-c9ec-5aa5-8331-a5a249b664cf,True,"[1446633, 1446633]",b0cfa94f-c9ec-5aa5-8331-a5a249b664cf,"[Guardians of Ga'Hoole, Xiaolin Showdown]",[A single tear escaped me as I left. I did hav...
3,e6e86e73-9a7b-58f2-a652-a17b4a1bcabf,True,"[1446633, 1446633]",e6e86e73-9a7b-58f2-a652-a17b4a1bcabf,"[Hetalia - Axis Powers, Warriors]","[""Ja."" Ludwig kept his gaze upon her, solidly...."
4,4fe541af-912e-5a86-81a5-94c6d3891509,True,"[1446633, 1446633]",4fe541af-912e-5a86-81a5-94c6d3891509,"[Hetalia - Axis Powers, Xiaolin Showdown]","[And he did. Slowly, hesitantly...but coming f..."


In [9]:
df_join_training_data = df_join_training_data.loc[:,~df_join_training_data.columns.duplicated()]

In [10]:
df_join_training_data.head()

,id,same,authors,fandoms,pair
0,6cced668-6e51-5212-873c-717f2bc91ce6,True,"[1446633, 1446633]","[Guardians of Ga'Hoole, Hetalia - Axis Powers]","[I shift a bit, warily letting my eyes dart fr..."
1,3c6c188a-db28-59aa-8c09-3d0f799ff579,True,"[1446633, 1446633]","[Guardians of Ga'Hoole, Warriors]","[I shift a bit, warily letting my eyes dart fr..."
2,b0cfa94f-c9ec-5aa5-8331-a5a249b664cf,True,"[1446633, 1446633]","[Guardians of Ga'Hoole, Xiaolin Showdown]",[A single tear escaped me as I left. I did hav...
3,e6e86e73-9a7b-58f2-a652-a17b4a1bcabf,True,"[1446633, 1446633]","[Hetalia - Axis Powers, Warriors]","[""Ja."" Ludwig kept his gaze upon her, solidly...."
4,4fe541af-912e-5a86-81a5-94c6d3891509,True,"[1446633, 1446633]","[Hetalia - Axis Powers, Xiaolin Showdown]","[And he did. Slowly, hesitantly...but coming f..."


In [11]:
df_join_training_data.dtypes

id         object
same         bool
authors    object
fandoms    object
pair       object
dtype: object

In [12]:
text = df_join_training_data.loc[0].pair

In [13]:
type(text), len(text)

(list, 2)

In [14]:
df_join_training_data[['text1','text2']] = pd.DataFrame(df_join_training_data.pair.tolist(), index= df_join_training_data.index)
df_join_training_data[['author1','author2']] = pd.DataFrame(df_join_training_data.authors.tolist(), index= df_join_training_data.index)

In [15]:
df_join_training_data = df_join_training_data.drop(columns=["pair", "fandoms","authors"])

In [16]:
df_join_training_data

,id,same,text1,text2,author1,author2
0,6cced668-6e51-5212-873c-717f2bc91ce6,True,"I shift a bit, warily letting my eyes dart fro...","""All will become one with Russia,"" he said, al...",1446633,1446633
1,3c6c188a-db28-59aa-8c09-3d0f799ff579,True,"I shift a bit, warily letting my eyes dart fro...","Suddenly, a piece of ice falls into the pit of...",1446633,1446633
2,b0cfa94f-c9ec-5aa5-8331-a5a249b664cf,True,A single tear escaped me as I left. I did have...,"got the Yang yoyo."" Kimiko pulled the other ha...",1446633,1446633
3,e6e86e73-9a7b-58f2-a652-a17b4a1bcabf,True,"""Ja."" Ludwig kept his gaze upon her, solidly. ...",SilverGray lll...YellowRagged llll...GrayMilli...,1446633,1446633
4,4fe541af-912e-5a86-81a5-94c6d3891509,True,"And he did. Slowly, hesitantly...but coming fr...","""Let""s go,"" Raimondo said and then started in ...",1446633,1446633
...,...,...,...,...,...,...
52596,3926a1bd-6d33-5513-a694-12d7590443d4,False,Enjoy! And time stands still beneath the air o...,"""That a challenge, dick?"" Austin smirks back a...",1940652,4353578
52597,17227d87-a3cb-5aca-b5fd-bb642e88b030,False,"""I forgive you,"" she blurted out looking strai...","This is a KNB fanfic. KNB doesn""t belong to me...",2581924,5085069
52598,0332dcaa-7230-508b-badb-820f9d8eebaa,False,"""What? Why? What""s happening? Where are we goi...","At the moment, Chris"" position on the swing se...",4819080,6983686
52599,f5e4884a-13fd-51fb-a880-053d6655d59a,False,"""Gah!"" I growled. ""I need a password to access...",As you can see a lot of things has happened si...,6070317,4369540


In [17]:
df_join_training_data.to_csv("../data/temp/data.csv", index=False)

#### Load csv

In [18]:
df_join_training_data = pd.read_csv("../data/temp/data.csv")

In [19]:
df_join_training_data

,id,same,text1,text2,author1,author2
0,6cced668-6e51-5212-873c-717f2bc91ce6,True,"I shift a bit, warily letting my eyes dart fro...","""All will become one with Russia,"" he said, al...",1446633,1446633
1,3c6c188a-db28-59aa-8c09-3d0f799ff579,True,"I shift a bit, warily letting my eyes dart fro...","Suddenly, a piece of ice falls into the pit of...",1446633,1446633
2,b0cfa94f-c9ec-5aa5-8331-a5a249b664cf,True,A single tear escaped me as I left. I did have...,"got the Yang yoyo."" Kimiko pulled the other ha...",1446633,1446633
3,e6e86e73-9a7b-58f2-a652-a17b4a1bcabf,True,"""Ja."" Ludwig kept his gaze upon her, solidly. ...",SilverGray lll...YellowRagged llll...GrayMilli...,1446633,1446633
4,4fe541af-912e-5a86-81a5-94c6d3891509,True,"And he did. Slowly, hesitantly...but coming fr...","""Let""s go,"" Raimondo said and then started in ...",1446633,1446633
...,...,...,...,...,...,...
52596,3926a1bd-6d33-5513-a694-12d7590443d4,False,Enjoy! And time stands still beneath the air o...,"""That a challenge, dick?"" Austin smirks back a...",1940652,4353578
52597,17227d87-a3cb-5aca-b5fd-bb642e88b030,False,"""I forgive you,"" she blurted out looking strai...","This is a KNB fanfic. KNB doesn""t belong to me...",2581924,5085069
52598,0332dcaa-7230-508b-badb-820f9d8eebaa,False,"""What? Why? What""s happening? Where are we goi...","At the moment, Chris"" position on the swing se...",4819080,6983686
52599,f5e4884a-13fd-51fb-a880-053d6655d59a,False,"""Gah!"" I growled. ""I need a password to access...",As you can see a lot of things has happened si...,6070317,4369540


In [20]:
texts = np.hstack([df_join_training_data['text1'].values, df_join_training_data['text2'].values])


In [21]:
dataset = df_join_training_data.sample(frac=1).reset_index(drop=True)

In [22]:
if len_dataset:
    dataset = dataset[:len_dataset]

##### Divide train / dev / test

In [23]:
train, test = train_test_split(dataset, test_size=0.15, random_state=sklearn_random)

In [24]:
train.same.value_counts()

True     903
False    797
Name: same, dtype: int64

In [25]:
test.same.value_counts()

True     174
False    126
Name: same, dtype: int64

In [26]:
train, dev = train_test_split(train, test_size=0.18, random_state=sklearn_random)

In [27]:
train.same.value_counts(),dev.same.value_counts(),test.same.value_counts()

(True     745
 False    649
 Name: same, dtype: int64,
 True     158
 False    148
 Name: same, dtype: int64,
 True     174
 False    126
 Name: same, dtype: int64)

In [28]:
train.to_csv("../data/temp/train.csv", index=False)
dev.to_csv("../data/temp/dev.csv", index=False)
test.to_csv("../data/temp/test.csv", index=False)